In [26]:
%pip install owslib shapely matplotlib

Note: you may need to restart the kernel to use updated packages.


### Imports

In [12]:
from owslib.wps import WebProcessingService, ComplexDataInput, monitorExecution
from owslib import wps
import matplotlib
from matplotlib import pyplot
from shapely.geometry import shape
import json
import pandas as pd
import rasterio
import logging
import sys
import io
from ipyleaflet import Map, GeoJSON, basemaps

In [13]:
# Set up owslib logging, to set relevant output
logger = logging.getLogger("owslib")
logger.setLevel(logging.WARNING)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.WARNING)
logger.addHandler(handler)

### Consts - Change these to affect the data retrieved

In [14]:
ENDPOINT="https://funcwps.ternlandscapes.org.au/wps/"
DATASET="smips"  # options are "smips" or "aet"
LAYER="totalbucket"  # options are "totalbucket" or "SMindex" for smips, or "ETa" for "aet".
STARTDATE="2007-01-01"
ENDDATE="2007-09-26"
# Enter your POLYGON as GeoJSON format.
MY_POLYGON='''{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              152.4076151068369,
              -27.740499600282057
            ],
            [
              153.4719537627558,
              -27.740499600282057
            ],
            [
              153.4719537627558,
              -27.093849063222414
            ],
            [
              152.4076151068369,
              -27.093849063222414
            ],
            [
              152.4076151068369,
              -27.740499600282057
            ]
          ]
        ],
        "type": "Polygon"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              153.04560361385302,
              -30.261677316183146
            ],
            [
              153.01223568656428,
              -30.274209977967658
            ],
            [
              152.97925359656017,
              -30.306757809371064
            ],
            [
              153.00516809584911,
              -30.388080106869154
            ],
            [
              153.11038986207154,
              -30.357268040135885
            ],
            [
              153.14690574743526,
              -30.313551938010704
            ],
            [
              153.12216917993214,
              -30.248449987166744
            ],
            [
              153.04560361385302,
              -30.261677316183146
            ]
          ]
        ],
        "type": "Polygon"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              150.9019211334824,
              -34.00680553492937
            ],
            [
              151.0874349541068,
              -34.00971216084313
            ],
            [
              151.31917860367787,
              -33.89045896204837
            ],
            [
              151.25957039365034,
              -33.692305214021
            ],
            [
              150.78621107872112,
              -33.642695168765655
            ],
            [
              150.71257740751076,
              -33.92537963226312
            ],
            [
              150.9019211334824,
              -34.00680553492937
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}'''

### Define a connection to the server

In [15]:
server = WebProcessingService(ENDPOINT, verbose=False)

In [16]:
print('WPS Identification type: %s' % server.identification.type)
print('WPS Identification title: %s' % server.identification.title)
print('WPS Identification abstract: %s' % server.identification.abstract)
for operation in server.operations:
    print('WPS Operation: %s' % operation.name)
for process in server.processes:
    print('WPS Process: identifier=%s title=%s' % (process.identifier, process.title))

WPS Identification type: WPS
WPS Identification title: TERN Landscapes Functional Data WPS Server
WPS Identification abstract: The Functional Data Processing Server from TERN Landscapes
WPS Operation: GetCapabilities
WPS Operation: DescribeProcess
WPS Operation: Execute
WPS Process: identifier=sleep title=Sleep Process
WPS Process: identifier=temporalDrill title=Temporal
WPS Process: identifier=subset1d title=SubSet 1D
WPS Process: identifier=subset2d title=SubSet 2D
WPS Process: identifier=stats2d title=Stats 2D
WPS Process: identifier=stats3d title=Stats 3D


In [17]:
process = server.describeprocess("stats3d")
print('WPS Process: identifier=%s' % process.identifier)
print('WPS Process: title=%s' % process.title)
print('WPS Process: abstract=%s' % process.abstract)
for input in process.dataInputs:
    print('Process input: identifier=%s, data type=%s, minOccurs=%d, maxOccurs=%d' % (input.identifier, input.dataType, input.minOccurs, input.maxOccurs))
for output in process.processOutputs:
    print('Process output: identifier=%s, data type=%s' % (output.identifier, output.dataType))

WPS Process: identifier=stats3d
WPS Process: title=Stats 3D
WPS Process: abstract=Return CSV file containing timeseries stats within given 2D polygons.
Process input: identifier=datasetId, data type=string, minOccurs=1, maxOccurs=1
Process input: identifier=startDate, data type=date, minOccurs=1, maxOccurs=1
Process input: identifier=endDate, data type=date, minOccurs=1, maxOccurs=1
Process input: identifier=polygon, data type=ComplexData, minOccurs=1, maxOccurs=1
Process output: identifier=download_link, data type=string
Process output: identifier=csv, data type=ComplexData


### Optional - Visualize shapes

In [18]:
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=[-25.128, 135.588], zoom=4)
m.add_layer(GeoJSON(data=json.loads(MY_POLYGON)))
m

Map(center=[-25.128, 135.588], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

### Set up to send request, first create inputs

In [19]:
poly_input = ComplexDataInput(MY_POLYGON, mimeType="application/vnd.geo+json", schema="http://geojson.org/geojson-spec.html#FeatureCollection")

### Trigger process to begin (process executes asynchronously)

In [20]:
resp = server.execute(
    "stats3d",
    mode="async",
    inputs=[("datasetId", f"{DATASET}:{LAYER}"), ("polygon", poly_input), ("startDate", STARTDATE), ("endDate", ENDDATE)],
    output=[("csv", False, "text/csv")]  # Get stats as embedded file in result
)

### Wait for process to complete (polls every 4 seconds)

In [21]:
while not resp.isComplete():
    print("Waiting for process to complete...")
    resp.checkStatus(sleepSecs=4)
    print(f"Status: {resp.status}")
print("Done")

Waiting for process to complete...
Status: ProcessSucceeded
Done


### Extract CSV data and render it

In [22]:
first_output = next(iter(resp.processOutputs))
data = "".join(d for d in first_output.data)
df = pd.read_csv(io.StringIO(data))
df

,None
